In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
%cd "/content/drive/MyDrive/Face_Recognition_Practice"

In [ ]:
!pip install face_recognition

In [ ]:
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [ ]:
def train(train_dir, model_save_path=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []

    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image, model="cnn")
            if len(face_bounding_boxes) != 1:
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)
                if verbose:
                  print("Image {}: Found face".format(img_path))
    n_neighbors = int(round(math.sqrt(len(X))))
    print("Chose n_neighbors automatically:", n_neighbors)
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)
    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)
    return knn_clf

In [ ]:
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))
    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img, model="cnn")
    if len(X_face_locations) == 0:
        return []
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]
    return [(pred, loc) if rec else (-1, loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]

In [ ]:
print("Training KNN classifier...")
classifier = train("sample", model_save_path="trained_knn_model.clf")
print("Training complete!")

In [ ]:
import threading

names = ["Person 1", "Person 2", "Person 3", "Person 4"]

# # STEP 2: Using the trained classifier, make predictions for unknown images
for image_file in os.listdir("unknown"):
    full_file_path = os.path.join("unknown", image_file)

    print("Looking for faces in {}".format(image_file))

    # Find all people in the image using a trained classifier model
    # Note: You can pass in either a classifier file name or a classifier model instance
    predictions = predict(full_file_path, model_path="trained_knn_model.clf", distance_threshold=0.4)

    # Print results on the console
    for index, (top, right, bottom, left) in predictions:
      if index != -1:
        print("- Found {} at ({}, {})".format(names[int(index)-1], left, top))
      else:
        print("unknwon")

    # Display results overlaid on an image
    # show_prediction_labels_on_image(os.path.join("unknown", image_file), predictions)

In [ ]:
from google.colab import files
files.download("trained_knn_model.clf")